# Question and Answer with RAG

In this notebook, we pass questions to the deployed function and see the answers from the LLM based on RAG. This can be done interactively via the ipywidget in the last cell. \
The list of tasks handled by the function are :
* Call the python helper functions script
* Connect to Elasticseach, Milvus or Datastax connection to search and retrieve the relevant documents
* Initialize the WML client
* Define a function for querying against the LLM
* Ask one of more questions of the LLM using the deployed AI service

### Contents

- [Pre-Requisite Libraries and Dependencies](#setup)
- [Import Parameter Set](#parameterimport)
- [Ask your Questions](#askquestions)
- [Ask your Questions (Interactively)](#askquestionsia)

<a id="setup"></a>
### Pre-Requisite Libraries and Dependencies
Download and import mandatory libraries and dependencies. 

Note : Some of the versions of the libraries may throw warnings after installation. These library versions are crucial for successful execution of the accelerator. Please ignore the warning/error and proceed with your execution. 

In [ ]:
!pip install ibm_watsonx_ai==1.3.26 | tail -n 1


In [ ]:
from ibm_watsonx_ai import APIClient
import os
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Markdown
import warnings
warnings.filterwarnings("ignore")

In [ ]:
project_id=os.environ['PROJECT_ID']
# Environment and host url
hostname = os.environ['RUNTIME_ENV_APSX_URL']

if hostname.endswith("cloud.ibm.com") == True:
    environment = "cloud"
    runtime_region = os.environ["RUNTIME_ENV_REGION"] 
else:
    environment = "on-prem"
    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()




<a id="parameterimport"></a>
### Import Parameter Sets, Credentials and Helper functions script.



The below cells imports the parameter set, credentials for watsonx.ai and helper function script. 

In [ ]:
try:
    filename = 'rag_helper_functions.py'
    wslib.download_file(filename)
    import rag_helper_functions
    print("rag_helper_functions imported from the project assets")
except NameError as e:
    print(str(e))
    print("If running watsonx.ai aaS on IBM Cloud, check that the first cell in the notebook contains a project token. If not, select the vertical ellipsis button from the notebook toolbar and `insert project token`. Also check that you have specified your ibm_api_key in the second code cell of the notebook")


In [ ]:
parameter_sets = ["RAG_parameter_set","RAG_advanced_parameter_set"]

parameters=rag_helper_functions.get_parameter_sets(wslib, parameter_sets)

In [ ]:
ibm_api_key=parameters['watsonx_ai_api_key']
if environment == "cloud":
    WML_SERVICE_URL=f"https://{runtime_region}.ml.cloud.ibm.com" 
    wml_credentials = {"apikey": ibm_api_key, "url": WML_SERVICE_URL}
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials = {"token": token,"instance_id" : "openshift","url": hostname}

### Set Watsonx.ai client
Below cell uses the watson machine learning credentials to create an API client to interact with the project and deployment space. 

In [ ]:
client = APIClient(wml_credentials)

deployment_id = parameters['wml_rag_deployment_id']
space_id = parameters['watsonx_ai_space_id']
client.set.default_space(space_id)

<a id="askquestions"></a>
### Ask your Questions
Enter your questions in the cell below

In [ ]:
questions = ["how to perform decision optimization?"]
query_filter=None
#query_filter={"metadata.title.keyword": {"query": "Adding connections to data sources in a project"}}
for q in questions:
    answer,documents,expert_answer, log_id=rag_helper_functions.query_llm(client, deployment_id, q,query_filter)
    rag_helper_functions.display_results(q, documents,True, answer)

### Display expert profile information based on the question. Optional

In [ ]:
if 'text' in expert_answer:
    print("\033[1mEXPERT PROFILE DETAILS\033[0m \n\nExpert Identifier:", expert_answer['expert_id'], \
    "\nExpert Name:", expert_answer['name'], \
    "\nExpert Email:", expert_answer['email'], \
    "\nExpert Text:", expert_answer['text'])
else:
    print(expert_answer)

<a id="askquestionsia"></a>
### Ask your Questions (interactively)

The following cell offers an interactive user interface for inputting questions. Clicking submit calls the deployed AI service and displays the answers returned along with a feedback section to provide feedback on the response.

For the dynamic UI to function correctly, ensure that iPywidgets are enabled in your Jupyter notebook. If the cell below does not display interactive output in a cloud environment, follow these steps:

    In your project, Go to "Manage" -> "Environments" -> "New Template" -> Enter a name for the new environment -> Edit the environment after creation -> Enable IPyWidgets in Notebook Extensions -> Click "Save".
    Update the environment of this notebook by going to "Information" (click on the 'i' at the right corner of this notebook) -> "Environment" -> Select your environment template.

In [ ]:
rag_helper_functions.qa_with_llm(client, deployment_id)

Use the filters to specify the the `Field` and `Value` for the field to filter out the result.
- e.g. `Field` = `metadata.title.keyword` and  `Value`= `SQL optimization (SPSS Modeler)` will filter the source documents title with value SQL optimization (SPSS Modeler) only and LLM will generate the response from those source documents.

**Note** Filters are not supported with datastax connections

**Sample Materials, provided under license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2024, 2025. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**